# Анализ воронки продаж Olist

**Цель проекта:** Проанализировать путь клиента от маркетингового лида (MQL) до заключения сделки (Closed Deal), рассчитать конверсию и визуализировать ключевые метрики.

In [5]:
# Импорт библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

# Настройка стилей для графиков
plt.style.use('ggplot')
sns.set_palette("Set2")
pd.set_option('display.max_columns', None)

# Загрузка данных
print("=" * 80)
print("ЗАГРУЗКА ДАННЫХ")
print("=" * 80)

mql_df = pd.read_csv('/content/olist_marketing_qualified_leads_dataset.csv')
deals_df = pd.read_csv('/content/olist_closed_deals_dataset.csv')

print(f"\n Marketing Qualified Leads: {df_leads.shape}")
print(df_leads.head())
print(f"\n Closed Deals: {df_deals.shape}")
print(df_deals.head())


ЗАГРУЗКА ДАННЫХ

 Marketing Qualified Leads: (8000, 4)
                             mql_id first_contact_date  \
0  dac32acd4db4c29c230538b72f8dd87d         2018-02-01   
1  8c18d1de7f67e60dbd64e3c07d7e9d5d         2017-10-20   
2  b4bc852d233dfefc5131f593b538befa         2018-03-22   
3  6be030b81c75970747525b843c1ef4f8         2018-01-22   
4  5420aad7fec3549a85876ba1c529bd84         2018-02-21   

                    landing_page_id          origin  
0  88740e65d5d6b056e0cda098e1ea6313          social  
1  007f9098284a86ee80ddeb25d53e0af8     paid_search  
2  a7982125ff7aa3b2054c6e44f9d28522  organic_search  
3  d45d558f0daeecf3cccdffe3c59684aa           email  
4  b48ec5f3b04e9068441002a19df93c6c  organic_search  

 Closed Deals: (842, 14)
                             mql_id                         seller_id  \
0  5420aad7fec3549a85876ba1c529bd84  2c43fb513632d29b3b58df74816f1b06   
1  a555fb36b9368110ede0f043dfc3b9a0  bbb7d7893a450660432ea6652310ebb7   
2  327174d3648a2d047e8940d7

### Первичный анализ данных (EDA)

In [6]:
# Первые строки MQL датасета
print("Маркетинговые лиды (MQL):")
mql_df.head()

Маркетинговые лиды (MQL):


,mql_id,first_contact_date,landing_page_id,origin
0,dac32acd4db4c29c230538b72f8dd87d,2018-02-01,88740e65d5d6b056e0cda098e1ea6313,social
1,8c18d1de7f67e60dbd64e3c07d7e9d5d,2017-10-20,007f9098284a86ee80ddeb25d53e0af8,paid_search
2,b4bc852d233dfefc5131f593b538befa,2018-03-22,a7982125ff7aa3b2054c6e44f9d28522,organic_search
3,6be030b81c75970747525b843c1ef4f8,2018-01-22,d45d558f0daeecf3cccdffe3c59684aa,email
4,5420aad7fec3549a85876ba1c529bd84,2018-02-21,b48ec5f3b04e9068441002a19df93c6c,organic_search


In [7]:
# Общая информация о MQL датасете
print("Информация о MQL датасете:")
mql_df.info()

Информация о MQL датасете:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   mql_id              8000 non-null   object
 1   first_contact_date  8000 non-null   object
 2   landing_page_id     8000 non-null   object
 3   origin              7940 non-null   object
dtypes: object(4)
memory usage: 250.1+ KB


In [8]:
# Проверка пропусков в MQL
print("Пропуски в MQL датасете:")
mql_df.isnull().sum()

Пропуски в MQL датасете:


,0
mql_id,0
first_contact_date,0
landing_page_id,0
origin,60


In [9]:
# Первые строки датасета с закрытыми сделками
print("Закрытые сделки (Closed Deals):")
deals_df.head()

Закрытые сделки (Closed Deals):


,mql_id,seller_id,sdr_id,sr_id,won_date,business_segment,lead_type,lead_behaviour_profile,has_company,has_gtin,average_stock,business_type,declared_product_catalog_size,declared_monthly_revenue
0,5420aad7fec3549a85876ba1c529bd84,2c43fb513632d29b3b58df74816f1b06,a8387c01a09e99ce014107505b92388c,4ef15afb4b2723d8f3d81e51ec7afefe,2018-02-26 19:58:54,pet,online_medium,cat,NaN,NaN,NaN,reseller,NaN,0.0
1,a555fb36b9368110ede0f043dfc3b9a0,bbb7d7893a450660432ea6652310ebb7,09285259593c61296eef10c734121d5b,d3d1e91a157ea7f90548eef82f1955e3,2018-05-08 20:17:59,car_accessories,industry,eagle,NaN,NaN,NaN,reseller,NaN,0.0
2,327174d3648a2d047e8940d7d15204ca,612170e34b97004b3ba37eae81836b4c,b90f87164b5f8c2cfa5c8572834dbe3f,6565aa9ce3178a5caf6171827af3a9ba,2018-06-05 17:27:23,home_appliances,online_big,cat,NaN,NaN,NaN,reseller,NaN,0.0
3,f5fee8f7da74f4887f5bcae2bafb6dd6,21e1781e36faf92725dde4730a88ca0f,56bf83c4bb35763a51c2baab501b4c67,d3d1e91a157ea7f90548eef82f1955e3,2018-01-17 13:51:03,food_drink,online_small,NaN,NaN,NaN,NaN,reseller,NaN,0.0
4,ffe640179b554e295c167a2f6be528e0,ed8cb7b190ceb6067227478e48cf8dde,4b339f9567d060bcea4f5136b9f5949e,d3d1e91a157ea7f90548eef82f1955e3,2018-07-03 20:17:45,home_appliances,industry,wolf,NaN,NaN,NaN,manufacturer,NaN,0.0


In [10]:
# Общая информация о датасете с закрытыми сделками
print("Информация о датасете с закрытыми сделками:")
deals_df.info()

Информация о датасете с закрытыми сделками:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 842 entries, 0 to 841
Data columns (total 14 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   mql_id                         842 non-null    object 
 1   seller_id                      842 non-null    object 
 2   sdr_id                         842 non-null    object 
 3   sr_id                          842 non-null    object 
 4   won_date                       842 non-null    object 
 5   business_segment               841 non-null    object 
 6   lead_type                      836 non-null    object 
 7   lead_behaviour_profile         665 non-null    object 
 8   has_company                    63 non-null     object 
 9   has_gtin                       64 non-null     object 
 10  average_stock                  66 non-null     object 
 11  business_type                  832 non-null    object 
 12  declar

In [11]:
# Проверка пропусков в deals
print("Пропуски в deals датасете:")
deals_df.isnull().sum()

Пропуски в deals датасете:


,0
mql_id,0
seller_id,0
sdr_id,0
sr_id,0
won_date,0
business_segment,1
lead_type,6
lead_behaviour_profile,177
has_company,779
has_gtin,778


**Наблюдения:**
- В датасете с MQL есть пропуски в столбце `origin`.
- В датасете с закрытыми сделками очень много пропусков в колонках, связанных с бизнес-информацией (`business_segment`, `lead_type`, `lead_behaviour_profile` и т.д.). Это нормально, так как, возможно, эта информация собиралась не для всех лидов. Также много нулей в финансовых столбцах.

### Подготовка и очистка данных

In [12]:
# Копируем, чтобы не менять оригинал
mql_clean = mql_df.copy()

In [13]:
# Заполняем пустые значения в 'origin' на 'unknown'
mql_clean['origin'] = mql_clean['origin'].fillna('unknown')

In [14]:
# Приводим дату к правильному формату
mql_clean['first_contact_date'] = pd.to_datetime(mql_clean['first_contact_date'])

In [15]:
# Проверяем результат
print("MQL после очистки:")
print(mql_clean.isnull().sum())
mql_clean.head()

MQL после очистки:
mql_id                0
first_contact_date    0
landing_page_id       0
origin                0
dtype: int64


,mql_id,first_contact_date,landing_page_id,origin
0,dac32acd4db4c29c230538b72f8dd87d,2018-02-01,88740e65d5d6b056e0cda098e1ea6313,social
1,8c18d1de7f67e60dbd64e3c07d7e9d5d,2017-10-20,007f9098284a86ee80ddeb25d53e0af8,paid_search
2,b4bc852d233dfefc5131f593b538befa,2018-03-22,a7982125ff7aa3b2054c6e44f9d28522,organic_search
3,6be030b81c75970747525b843c1ef4f8,2018-01-22,d45d558f0daeecf3cccdffe3c59684aa,email
4,5420aad7fec3549a85876ba1c529bd84,2018-02-21,b48ec5f3b04e9068441002a19df93c6c,organic_search


In [16]:
# --- Обработка датасета с закрытыми сделками ---
deals_clean = deals_df.copy()

In [17]:
# Приводим дату сделки к правильному формату
deals_clean['won_date'] = pd.to_datetime(deals_clean['won_date'])

In [18]:
# Заполняем пропуски в категориальных признаках значением 'unknown'
# Создадим список колонок, которые хотим обработать
cat_cols_to_fill = ['business_segment', 'lead_type', 'lead_behaviour_profile', 'business_type']
for col in cat_cols_to_fill:
    deals_clean[col] = deals_clean[col].fillna('unknown')

In [19]:
# Проверяем результат
print("Deals после очистки:")
print(deals_clean.isnull().sum())
deals_clean.head()

Deals после очистки:
mql_id                             0
seller_id                          0
sdr_id                             0
sr_id                              0
won_date                           0
business_segment                   0
lead_type                          0
lead_behaviour_profile             0
has_company                      779
has_gtin                         778
average_stock                    776
business_type                      0
declared_product_catalog_size    773
declared_monthly_revenue           0
dtype: int64


,mql_id,seller_id,sdr_id,sr_id,won_date,business_segment,lead_type,lead_behaviour_profile,has_company,has_gtin,average_stock,business_type,declared_product_catalog_size,declared_monthly_revenue
0,5420aad7fec3549a85876ba1c529bd84,2c43fb513632d29b3b58df74816f1b06,a8387c01a09e99ce014107505b92388c,4ef15afb4b2723d8f3d81e51ec7afefe,2018-02-26 19:58:54,pet,online_medium,cat,NaN,NaN,NaN,reseller,NaN,0.0
1,a555fb36b9368110ede0f043dfc3b9a0,bbb7d7893a450660432ea6652310ebb7,09285259593c61296eef10c734121d5b,d3d1e91a157ea7f90548eef82f1955e3,2018-05-08 20:17:59,car_accessories,industry,eagle,NaN,NaN,NaN,reseller,NaN,0.0
2,327174d3648a2d047e8940d7d15204ca,612170e34b97004b3ba37eae81836b4c,b90f87164b5f8c2cfa5c8572834dbe3f,6565aa9ce3178a5caf6171827af3a9ba,2018-06-05 17:27:23,home_appliances,online_big,cat,NaN,NaN,NaN,reseller,NaN,0.0
3,f5fee8f7da74f4887f5bcae2bafb6dd6,21e1781e36faf92725dde4730a88ca0f,56bf83c4bb35763a51c2baab501b4c67,d3d1e91a157ea7f90548eef82f1955e3,2018-01-17 13:51:03,food_drink,online_small,unknown,NaN,NaN,NaN,reseller,NaN,0.0
4,ffe640179b554e295c167a2f6be528e0,ed8cb7b190ceb6067227478e48cf8dde,4b339f9567d060bcea4f5136b9f5949e,d3d1e91a157ea7f90548eef82f1955e3,2018-07-03 20:17:45,home_appliances,industry,wolf,NaN,NaN,NaN,manufacturer,NaN,0.0


Комментарий:
- Мы заполнили пропуски в столбце ` origin` и категориальных столбцах значением `'unknown'`. Это позволяет сохранить все строки для анализа, явно обозначив, что информация отсутствует.
- Нули в финансовых столбцах оставили без изменений, так как они могут означать отсутствие дохода или товаров.


### Расчет конверсии и метрик воронки

In [20]:
# Общее количество MQL (Вход в воронку)
total_mql = len(mql_clean)
print(f"Всего маркетинговых лидов (MQL): {total_mql}")

Всего маркетинговых лидов (MQL): 8000


In [21]:
# Количество закрытых сделок (Выход из воронки)
# Уникальные mql_id в deals_clean, так как один mql может привести к сделке только один раз
total_deals = deals_clean['mql_id'].nunique()
print(f"Всего закрытых сделок (Closed Deals): {total_deals}")


Всего закрытых сделок (Closed Deals): 842


In [22]:
# Общая конверсия из MQL в сделку
if total_mql > 0:
    overall_conversion_rate = (total_deals / total_mql) * 100
    print(f"Общая конверсия MQL -> Closed Deal: {overall_conversion_rate:.2f}%")
else:
    print("Нет данных по MQL для расчета конверсии.")

Общая конверсия MQL -> Closed Deal: 10.53%


Создадим объединенный датафрейм, чтобы анализировать, какие маркетинговые параметры (источник) влияют на конверсию

In [23]:
merged_df = mql_clean.merge(deals_clean[['mql_id', 'seller_id', 'business_segment', 'lead_type', 'business_type']],
                             on='mql_id', how='left', suffixes=('_mql', '_deal'))


In [24]:
# Создаем колонку-флаг, была ли сделка закрыта
merged_df['is_converted'] = ~merged_df['seller_id'].isna()

In [25]:
# Смотрим на результат
print("Объединенный датафрейм (первые строки):")
merged_df.head()

Объединенный датафрейм (первые строки):


,mql_id,first_contact_date,landing_page_id,origin,seller_id,business_segment,lead_type,business_type,is_converted
0,dac32acd4db4c29c230538b72f8dd87d,2018-02-01,88740e65d5d6b056e0cda098e1ea6313,social,NaN,NaN,NaN,NaN,False
1,8c18d1de7f67e60dbd64e3c07d7e9d5d,2017-10-20,007f9098284a86ee80ddeb25d53e0af8,paid_search,NaN,NaN,NaN,NaN,False
2,b4bc852d233dfefc5131f593b538befa,2018-03-22,a7982125ff7aa3b2054c6e44f9d28522,organic_search,NaN,NaN,NaN,NaN,False
3,6be030b81c75970747525b843c1ef4f8,2018-01-22,d45d558f0daeecf3cccdffe3c59684aa,email,NaN,NaN,NaN,NaN,False
4,5420aad7fec3549a85876ba1c529bd84,2018-02-21,b48ec5f3b04e9068441002a19df93c6c,organic_search,2c43fb513632d29b3b58df74816f1b06,pet,online_medium,reseller,True


In [26]:
# Проверим количество конвертированных лидов
print(f"Количество конвертированных лидов в объединенном датафрейме: {merged_df['is_converted'].sum()}")
# Должно совпасть с total_deals

Количество конвертированных лидов в объединенном датафрейме: 842


Анализ конверсии по источникам трафика (`origin`)

In [27]:
# Группировка по источнику трафика
conversion_by_origin = merged_df.groupby('origin').agg(
    total_leads=('mql_id', 'count'),
    converted_leads=('is_converted', 'sum')
).reset_index()

In [29]:
# Расчет конверсии
conversion_by_origin['conversion_rate'] = (conversion_by_origin['converted_leads'] / conversion_by_origin['total_leads']) * 100

# Сортировка по конверсии
conversion_by_origin = conversion_by_origin.sort_values('conversion_rate', ascending=False).reset_index(drop=True)

print("Конверсия по источникам трафика:")
conversion_by_origin

Конверсия по источникам трафика:


,origin,total_leads,converted_leads,conversion_rate
0,unknown,1159,193,16.652286
1,paid_search,1586,195,12.295082
2,organic_search,2296,271,11.803136
3,direct_traffic,499,56,11.222445
4,referral,284,24,8.450704
5,social,1350,75,5.555556
6,display,118,6,5.084746
7,other_publicities,65,3,4.615385
8,email,493,15,3.042596
9,other,150,4,2.666667


Анализ конверсии по бизнес-сегменту (для тех, кто дошел до сделки)

In [30]:
# Берем только конвертированные лиды для анализа их параметров
converted_df = deals_clean.copy()

# Группировка по бизнес-сегменту
conversion_by_segment = converted_df.groupby('business_segment').agg(
    total_deals=('mql_id', 'count')
).reset_index().sort_values('total_deals', ascending=False)

print("Распределение закрытых сделок по бизнес-сегментам:")
conversion_by_segment.head(10)

Распределение закрытых сделок по бизнес-сегментам:


,business_segment,total_deals
17,home_decor,105
15,health_beauty,93
6,car_accessories,77
19,household_utilities,71
8,construction_tools_house_garden,69
1,audio_video_electronics,64
7,computers,34
25,pet,30
11,food_supplement,28
10,food_drink,26


### Визуализация

***1. Воронка продаж***

In [31]:
fig = go.Figure(go.Funnel(
    y = ['Маркетинговые лиды (MQL)', 'Закрытые сделки'],
    x = [total_mql, total_deals],
    textinfo = "value+percent initial"))

fig.update_layout(title="Воронка продаж Olist (MQL -> Closed Deal)",
                  funnelgap = 0.2)
fig.show()


***2. Распределение MQL по источникам трафика***

In [32]:
fig = px.pie(conversion_by_origin, values='total_leads', names='origin',
             title='Распределение маркетинговых лидов по источникам трафика',
             hole=0.3)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

***3. Конверсия по источникам трафика (столбчатая диаграмма)***

In [33]:
fig = px.bar(conversion_by_origin, x='origin', y='conversion_rate',
             title='Конверсия MQL -> Closed Deal по источникам трафика',
             labels={'origin': 'Источник трафика', 'conversion_rate': 'Конверсия (%)'},
             text='conversion_rate')
fig.update_traces(texttemplate='%{text:.2f}%', textposition='outside')
fig.update_layout(xaxis_tickangle=-45)
fig.show()

***4. Топ-10 бизнес-сегментов по количеству закрытых сделок***

In [34]:
fig = px.bar(conversion_by_segment.head(10), x='total_deals', y='business_segment',
             title='Топ-10 бизнес-сегментов по количеству закрытых сделок',
             labels={'total_deals': 'Количество сделок', 'business_segment': 'Бизнес-сегмент'},
             orientation='h', text='total_deals')
fig.update_traces(textposition='outside')
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()

In [36]:
import plotly.graph_objects as go

# Данные по сегментам
segments = {
    'home_decor': 105,
    'health_beauty': 93,
    'car_accessories': 77,
    'household_utilities': 71,
    'construction_tools_house_garden': 69,
    'audio_video_electronics': 64,
    'computers': 34,
    'pet': 30,
    'food_supplement': 28,
    'food_drink': 26
}

# Создаем улучшенную визуализацию
fig = go.Figure(data=[
    go.Bar(
        x=list(segments.values()),
        y=list(segments.keys()),
        orientation='h',
        marker=dict(
            color=['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#3B8F7F',
                   '#7E5A9B', '#D9504B', '#3A6EA5', '#D98C5F', '#5C7A5C'],
            line=dict(color='white', width=2)
        ),
        text=list(segments.values()),
        textposition='outside',
        textfont=dict(size=14, color='black')
    )
])

fig.update_layout(
    title={
        'text': 'Топ-10 бизнес-сегментов по количеству сделок',
        'x': 0.5,
        'font': {'size': 24, 'family': 'Arial Black'}
    },
    xaxis_title='Количество сделок',
    yaxis_title='Бизнес-сегмент',
    height=600,
    width=900,
    showlegend=False,
    plot_bgcolor='white',
    xaxis=dict(
        gridcolor='lightgray',
        range=[0, 120]
    )
)

fig.show()

# Расчет долей
total_deals = sum(segments.values())
print(f"\n Анализ долей рынка:")
for segment, deals in segments.items():
    share = (deals / total_deals) * 100
    print(f"{segment:35} {deals:3} сделок ({share:.1f}%)")

print(f"\n Топ-3 сегмента составляют {sum(list(segments.values())[:3])/total_deals*100:.1f}% всех сделок")


 Анализ долей рынка:
home_decor                          105 сделок (17.6%)
health_beauty                        93 сделок (15.6%)
car_accessories                      77 сделок (12.9%)
household_utilities                  71 сделок (11.9%)
construction_tools_house_garden      69 сделок (11.6%)
audio_video_electronics              64 сделок (10.7%)
computers                            34 сделок (5.7%)
pet                                  30 сделок (5.0%)
food_supplement                      28 сделок (4.7%)
food_drink                           26 сделок (4.4%)

 Топ-3 сегмента составляют 46.1% всех сделок
